In [0]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [7]:
# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)
print(x, w, b)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3
print(y)

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1
print(x, w, b)

tensor(1., requires_grad=True) tensor(2., requires_grad=True) tensor(3., requires_grad=True)
tensor(5., grad_fn=<AddBackward0>)
tensor(2.)
tensor(1.)
tensor(1.)
tensor(1., requires_grad=True) tensor(2., requires_grad=True) tensor(3., requires_grad=True)


In [36]:
# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)
print("\nx=", x.shape, "\n", x)
print("\ny=", y.shape, "\n", y)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('\nw: ', w.size(), "\n", linear.weight)
print ('\nb: ', b.size(), "\n", linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)
print("\nPred=", pred.size(), "\n", pred)

pred = linear(x) - linear.bias
print("\ny-b=\n", pred)
print("\nx*w=\n", x.mm(linear.weight.t()))

# Compute loss.
loss = criterion(pred, y)
print('\nloss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('\ndL/dw: ', linear.weight.grad) 
print ('\ndL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('\nloss after 1 step optimization: ', loss.item())


x= torch.Size([10, 3]) 
 tensor([[-1.4880, -0.3077, -0.8820],
        [ 0.0460, -0.1332,  0.5862],
        [-0.5559, -0.2460,  0.2371],
        [-0.5387, -0.4273,  0.1445],
        [-0.2552,  1.0756,  0.0940],
        [ 2.1430,  0.5615,  1.5352],
        [ 0.3507,  1.4397,  0.0123],
        [ 0.9934, -0.5560,  0.5396],
        [-1.2338,  0.3468,  0.0433],
        [-1.5276, -0.5685,  2.0800]])

y= torch.Size([10, 2]) 
 tensor([[ 6.2137e-01, -3.2991e-02],
        [-3.0769e-01,  7.7710e-01],
        [ 1.6067e-01, -2.7794e-01],
        [ 1.7748e+00,  4.8085e-01],
        [-4.4364e-01,  1.3769e+00],
        [-8.0836e-01, -2.5529e-04],
        [-1.1920e+00, -1.3232e-01],
        [-1.7537e+00, -9.6403e-01],
        [ 1.3902e+00, -2.5871e+00],
        [-1.0053e+00,  1.2737e+00]])

w:  torch.Size([]) 
 Parameter containing:
tensor([[-0.3682, -0.4846, -0.3882],
        [-0.3856,  0.4483,  0.4432]], requires_grad=True)

b:  torch.Size([]) 
 Parameter containing:
tensor([0.5706, 0.1148], requires

In [38]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])
print(x)

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
print(y)

# Convert the torch tensor to a numpy array.
z = y.numpy()
print(z)

[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]])
[[1 2]
 [3 4]]


In [47]:
# ================================================================== #
#                         4. Input pipline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
i = 0
for images, labels in train_loader:
    # Training code should be written here.
    #pass
    i += 1

print(i)

Files already downloaded and verified
torch.Size([3, 32, 32])
6
782


In [48]:
# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth

  0%|          | 0.00/44.7M [00:00<?, ?B/s]
 11%|█         | 4.82M/44.7M [00:00<00:00, 50.5MB/s]
 46%|████▌     | 20.6M/44.7M [00:00<00:00, 63.8MB/s]
 80%|████████  | 35.9M/44.7M [00:00<00:00, 77.9MB/s]
100%|██████████| 44.7M/44.7M [00:00<00:00, 130MB/s] 

torch.Size([64, 100])


In [49]:
# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

<All keys matched successfully>